<a href="https://colab.research.google.com/github/vasudevabhat0605/Amazon-Review-Sentiment-Analysis/blob/main/sentiment_analysis_using_svm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scikit-learn
!pip install vaderSentiment
!pip install contractions
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
vs = SentimentIntensityAnalyzer()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
import re
import contractions
import unicodedata
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

     |████████████████████████████████| 133kB 5.4MB/s 
     |████████████████████████████████| 286kB 5.6MB/s 
     |████████████████████████████████| 327kB 7.5MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85404 sha256=c460f8e3ae7a8c28f5a936eed6b401d2576cb51bec7d60b069ef9892a95ff4ab
  Stored in directory: /root/.cache/pip/wheels/3a/03/34/77e3ece0bba8b86bfac88a79f923b36d805cad63caeba38842
Successfully built pyahocorasick
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [2]:
df = pd.read_csv("/content/drive/MyDrive/major_p_datasets/AllProductReviews.csv",nrows=5000)
print(df.shape)
df.ReviewTitle = df.ReviewTitle.str.replace('\n', '')
df.ReviewBody = df.ReviewBody.str.replace('\n', '')
df.head()

(5000, 4)


,ReviewTitle,ReviewBody,ReviewStar,Product
0,Honest review of an edm music lover,No doubt it has a great bass and to a great ex...,3,boAt Rockerz 255
1,Unreliable earphones with high cost,"This earphones are unreliable, i bought it be...",1,boAt Rockerz 255
2,Really good and durable.,"i bought itfor 999,I purchased it second time,...",4,boAt Rockerz 255
3,stopped working in just 14 days,Its sound quality is adorable. overall it was ...,1,boAt Rockerz 255
4,Just Awesome Wireless Headphone under 1000...😉,Its Awesome... Good sound quality & 8-9 hrs ba...,5,boAt Rockerz 255


In [3]:
lemmatizer = WordNetLemmatizer()
tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')
stopword_list.remove('no')
stopword_list.remove('not')
len(stopword_list)

def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

def expand_contractions(text):
    expanded_text = contractions.fix(text)
    return expanded_text

def remove_special_characters(text, remove_digits=True):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, ' ', text)
    return text

def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

In [4]:
df.ReviewTitle	= df.ReviewTitle.apply(lambda x:x.lower())
df.ReviewBody = df.ReviewBody.apply(lambda x:x.lower())

df.ReviewTitle = df.ReviewTitle.apply(remove_accented_chars)
df.ReviewBody = df.ReviewBody.apply(remove_accented_chars)

df.ReviewTitle = df.ReviewTitle.apply(expand_contractions)
df.ReviewBody = df.ReviewBody.apply(expand_contractions)

df.ReviewTitle = df.ReviewTitle.apply(remove_special_characters)
df.ReviewBody = df.ReviewBody.apply(remove_special_characters)

df.ReviewTitle = df.ReviewTitle.apply(remove_stopwords)
df.ReviewBody = df.ReviewBody.apply(remove_stopwords)

df['ReviewTitle_token'] = df.ReviewTitle.apply(lambda x: word_tokenize(x))
df['ReviewBody_token'] = df.ReviewBody.apply(lambda x: word_tokenize(x))

df['ReviewTitle_lem'] = df.ReviewTitle_token.apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
df['ReviewBody_lem'] = df.ReviewBody_token.apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

def listToString(s):  
    
    # initialize an empty string 
    str1 = " " 
    
    # return string   
    return (str1.join(s)) 

df['ReviewTitle_new'] = df.ReviewTitle_lem.apply(listToString)
df['ReviewBody_new'] = df.ReviewBody_lem.apply(listToString)

new_df = df[['ReviewTitle_new','ReviewBody_new','Product']]


new_df.to_csv('AllProductReviews.csv',index=True,encoding='utf-8')
new_df.head()

,ReviewTitle_new,ReviewBody_new,Product
0,honest review edm music lover,no doubt great bass great extent noise cancell...,boAt Rockerz 255
1,unreliable earphone high cost,earphone unreliable bought day meanwhile right...,boAt Rockerz 255
2,really good durable,bought itfor purchased second time gifted firs...,boAt Rockerz 255
3,stopped working day,sound quality adorable overall good week stopp...,boAt Rockerz 255
4,awesome wireless headphone,awesome good sound quality hr battery life waw...,boAt Rockerz 255


In [5]:
df['compound'] = df['ReviewBody'].apply(lambda x: vs.polarity_scores(x)['compound'])
df['comp_score'] = df['compound'].apply(lambda c: 'positive' if c>=0 else 'negative' )

In [6]:
x = new_df['ReviewBody_new'].values
y = df['ReviewStar'].values
x.astype(str)
y.astype(str)
x_train, x_test, y_train, y_test=train_test_split(x, y, test_size=0.1, random_state=0)

In [7]:
print(x_train.shape)
print(x_test.shape)
np.unique(y_train,return_counts=True)
np.unique(y_test,return_counts=True)

(4500,)
(500,)


(array([1, 2, 3, 4, 5]), array([ 84,  34,  63, 129, 190]))

In [8]:
vect = TfidfVectorizer()
x_train = vect.fit_transform(x_train)
x_test = vect.transform(x_test)

In [9]:
from sklearn.svm import SVC
model = SVC( )
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
y_pred

array([4, 5, 5, 1, 5, 1, 5, 5, 5, 5, 3, 4, 1, 1, 5, 4, 5, 4, 1, 5, 4, 4,
       5, 4, 1, 4, 5, 1, 5, 5, 1, 4, 5, 5, 4, 5, 5, 4, 4, 5, 5, 5, 3, 5,
       4, 1, 4, 5, 4, 5, 4, 1, 5, 5, 5, 1, 5, 5, 5, 5, 1, 5, 4, 5, 4, 4,
       1, 4, 3, 5, 5, 3, 5, 5, 5, 5, 1, 1, 1, 4, 1, 3, 5, 4, 5, 5, 1, 5,
       1, 5, 5, 4, 5, 5, 5, 5, 1, 5, 5, 4, 4, 4, 5, 5, 5, 1, 3, 5, 3, 1,
       5, 1, 1, 1, 5, 5, 5, 1, 4, 5, 5, 5, 1, 5, 5, 4, 5, 1, 5, 1, 4, 5,
       5, 4, 4, 5, 4, 4, 5, 5, 1, 4, 5, 5, 5, 5, 4, 5, 5, 5, 4, 4, 5, 1,
       1, 5, 5, 5, 5, 3, 5, 1, 5, 5, 5, 3, 5, 5, 1, 1, 5, 5, 5, 1, 5, 4,
       1, 5, 2, 5, 5, 5, 5, 5, 5, 3, 5, 4, 1, 1, 1, 5, 4, 5, 1, 5, 4, 1,
       5, 5, 5, 5, 5, 5, 1, 1, 5, 5, 5, 4, 5, 5, 5, 1, 5, 4, 5, 4, 5, 3,
       1, 5, 1, 5, 1, 1, 5, 5, 5, 4, 4, 4, 5, 5, 1, 5, 5, 4, 4, 4, 5, 5,
       5, 1, 1, 5, 5, 5, 5, 1, 4, 1, 4, 1, 1, 5, 3, 4, 5, 1, 1, 5, 5, 5,
       5, 3, 5, 5, 5, 5, 4, 5, 1, 5, 5, 5, 1, 5, 5, 1, 4, 5, 5, 3, 4, 1,
       5, 5, 5, 5, 1, 5, 5, 5, 3, 4, 1, 1, 4, 4, 5,

In [10]:
y_test

array([3, 5, 4, 1, 3, 5, 5, 5, 3, 3, 3, 5, 1, 3, 5, 3, 5, 1, 5, 4, 4, 5,
       4, 4, 1, 3, 4, 1, 5, 5, 1, 4, 5, 4, 5, 5, 4, 3, 4, 5, 4, 4, 2, 3,
       2, 2, 4, 5, 1, 3, 4, 2, 4, 5, 5, 1, 4, 5, 5, 5, 1, 4, 3, 5, 5, 3,
       1, 5, 5, 5, 5, 5, 3, 1, 4, 5, 1, 1, 1, 4, 1, 2, 4, 3, 5, 4, 2, 5,
       1, 5, 5, 4, 4, 5, 4, 5, 3, 5, 3, 1, 4, 5, 4, 5, 5, 1, 2, 3, 1, 1,
       5, 1, 3, 1, 3, 5, 3, 2, 1, 5, 5, 5, 3, 5, 5, 3, 5, 3, 5, 1, 4, 5,
       5, 4, 3, 5, 5, 3, 4, 4, 1, 2, 5, 4, 5, 5, 5, 3, 5, 5, 4, 2, 5, 1,
       1, 3, 5, 5, 5, 3, 2, 3, 4, 2, 4, 4, 5, 5, 1, 1, 5, 5, 4, 1, 5, 4,
       5, 5, 2, 5, 4, 4, 5, 5, 5, 4, 4, 5, 1, 1, 3, 5, 4, 4, 2, 5, 4, 2,
       3, 5, 4, 5, 5, 4, 2, 1, 4, 5, 4, 3, 5, 5, 1, 1, 4, 5, 4, 2, 3, 3,
       1, 4, 1, 4, 2, 1, 4, 5, 5, 4, 4, 5, 5, 5, 1, 5, 4, 3, 4, 4, 4, 4,
       5, 4, 2, 5, 5, 5, 5, 3, 4, 1, 3, 1, 1, 4, 1, 4, 5, 5, 1, 5, 1, 5,
       5, 2, 4, 4, 5, 4, 5, 5, 1, 5, 5, 5, 1, 1, 5, 1, 3, 4, 5, 3, 3, 5,
       1, 5, 5, 4, 4, 5, 5, 5, 4, 4, 1, 2, 1, 2, 5,

In [11]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
accuracy_score(y_pred,y_test)

0.542

In [12]:
confusion_matrix(y_pred,y_test)

array([[ 63,  16,  12,   3,   6],
       [  0,   1,   0,   0,   0],
       [  4,   6,   9,   3,   2],
       [  9,   6,  25,  42,  26],
       [  8,   5,  17,  81, 156]])

In [13]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           1       0.75      0.63      0.68       100
           2       0.03      1.00      0.06         1
           3       0.14      0.38      0.21        24
           4       0.33      0.39      0.35       108
           5       0.82      0.58      0.68       267

    accuracy                           0.54       500
   macro avg       0.41      0.60      0.40       500
weighted avg       0.67      0.54      0.59       500



In [14]:
x_train, x_test, y_train, y_test=train_test_split(x, y, test_size=0.1, random_state=0)
text_model = Pipeline([('vect',TfidfVectorizer( )),('model',SVC( ))])
text_model.fit(x_train,y_train)

Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('model',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', de

In [15]:
y_pred = text_model.predict(x_test)
y_pred

array([4, 5, 5, 1, 5, 1, 5, 5, 5, 5, 3, 4, 1, 1, 5, 4, 5, 4, 1, 5, 4, 4,
       5, 4, 1, 4, 5, 1, 5, 5, 1, 4, 5, 5, 4, 5, 5, 4, 4, 5, 5, 5, 3, 5,
       4, 1, 4, 5, 4, 5, 4, 1, 5, 5, 5, 1, 5, 5, 5, 5, 1, 5, 4, 5, 4, 4,
       1, 4, 3, 5, 5, 3, 5, 5, 5, 5, 1, 1, 1, 4, 1, 3, 5, 4, 5, 5, 1, 5,
       1, 5, 5, 4, 5, 5, 5, 5, 1, 5, 5, 4, 4, 4, 5, 5, 5, 1, 3, 5, 3, 1,
       5, 1, 1, 1, 5, 5, 5, 1, 4, 5, 5, 5, 1, 5, 5, 4, 5, 1, 5, 1, 4, 5,
       5, 4, 4, 5, 4, 4, 5, 5, 1, 4, 5, 5, 5, 5, 4, 5, 5, 5, 4, 4, 5, 1,
       1, 5, 5, 5, 5, 3, 5, 1, 5, 5, 5, 3, 5, 5, 1, 1, 5, 5, 5, 1, 5, 4,
       1, 5, 2, 5, 5, 5, 5, 5, 5, 3, 5, 4, 1, 1, 1, 5, 4, 5, 1, 5, 4, 1,
       5, 5, 5, 5, 5, 5, 1, 1, 5, 5, 5, 4, 5, 5, 5, 1, 5, 4, 5, 4, 5, 3,
       1, 5, 1, 5, 1, 1, 5, 5, 5, 4, 4, 4, 5, 5, 1, 5, 5, 4, 4, 4, 5, 5,
       5, 1, 1, 5, 5, 5, 5, 1, 4, 1, 4, 1, 1, 5, 3, 4, 5, 1, 1, 5, 5, 5,
       5, 3, 5, 5, 5, 5, 4, 5, 1, 5, 5, 5, 1, 5, 5, 1, 4, 5, 5, 3, 4, 1,
       5, 5, 5, 5, 1, 5, 5, 5, 3, 4, 1, 1, 4, 4, 5,

In [16]:
accuracy_score(y_pred,y_test)

0.542

In [17]:
text = df['ReviewBody'][10]
text_model.predict([text])

array([4])

In [18]:
!pip install joblib
import pickle
import joblib
joblib.dump(text_model,'sentiment analysis')

['sentiment analysis']